## Question 6: Association Rules - BAKERY
-  Use dataset 75000 (select one file in this folder that is suitable for you) to build the model to identify sets of items that are frequently bought together (In both two cases: use Id and use Flavor and Food name (in goods.csv)).
- Read more information here: http://users.csc.calpoly.edu/~dekhtyar/466-Spring2018/labs/lab01.html

# 0.Working environment preparation & Package import

In [1]:
# !apt-get update

In [2]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
# !tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
# !pip install -q findspark
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

In [4]:
# %cd "/content/gdrive/MyDrive/LDS9_K272_ONLINE_DamThiMinhPhuong/LDS9_K272_DamThiMinhPhuong_Cuoi_ky"

In [5]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

In [6]:
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.fpm import FPGrowth

In [7]:
spark = SparkSession.builder.appName('Q6_association_rules').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/21 17:33:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1. Data preprocessing

In [8]:
# Load data set 75000-out1.csv
df = pd.read_csv('data/75000/75000-out1.csv', sep='delimiter', header=None)

/opt/homebrew/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [9]:
df.head()

,0
0,"1, 11, 21"
1,"2, 7, 11, 37, 45"
2,"3, 3, 33, 42"
3,"4, 5, 12, 17, 47"
4,"5, 6, 18, 42"


In [10]:
df['order_id'] = df.index + 1
df.columns =['items', 'order_id']

In [11]:
df.head()

,items,order_id
0,"1, 11, 21",1
1,"2, 7, 11, 37, 45",2
2,"3, 3, 33, 42",3
3,"4, 5, 12, 17, 47",4
4,"5, 6, 18, 42",5


In [12]:
# Split string theo ',' -> đổi sang kiểu int -> list of int
df['items'] = df['items'].apply(lambda x: list(map(int, x.split(','))))

In [13]:
# Lấy unique (loại bỏ duplicated items)
df['items'] = df['items'].apply(lambda x: list(set(x)))

In [14]:
df.head()

,items,order_id
0,"[1, 11, 21]",1
1,"[2, 37, 7, 11, 45]",2
2,"[33, 42, 3]",3
3,"[4, 5, 12, 47, 17]",4
4,"[18, 42, 5, 6]",5


In [15]:
# convert pandas dataframe to pyspark dataframe
df_schema = StructType([StructField("items", ArrayType(elementType=IntegerType()), True)\
                       ,StructField("order_id", IntegerType(), True)])
data = spark.createDataFrame(df, schema=df_schema)

In [16]:
data.show(5)

+------------------+--------+
|             items|order_id|
+------------------+--------+
|       [1, 11, 21]|       1|
|[2, 37, 7, 11, 45]|       2|
|       [33, 42, 3]|       3|
|[4, 5, 12, 47, 17]|       4|
|    [18, 42, 5, 6]|       5|
+------------------+--------+
only showing top 5 rows



# 2. Build model 

## --- Use Id

In [17]:
# Build model & fit
fpGrowth = FPGrowth(itemsCol='items', minSupport=0.03, minConfidence=0.03)
model = fpGrowth.fit(data)

In [18]:
# Get frequent items
model.freqItemsets.sort('freq', ascending=False).show(10)

/Users/elliedam/spark/spark-3.2.0-bin-hadoop2.7/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-----+----+
|items|freq|
+-----+----+
|  [7]|8194|
| [45]|7701|
| [28]|7557|
| [18]|6988|
|  [4]|6949|
| [35]|6944|
| [42]|6872|
| [22]|6784|
| [27]|6734|
| [14]|6363|
+-----+----+
only showing top 10 rows



In [19]:
# Prediction
predictions = model.transform(data)
predictions.select('order_id', 'items', 'prediction').show(10)

+--------+--------------------+--------------------+
|order_id|               items|          prediction|
+--------+--------------------+--------------------+
|       1|         [1, 11, 21]|     [37, 45, 7, 19]|
|       2|  [2, 37, 7, 11, 45]| [15, 46, 0, 32, 16]|
|       3|         [33, 42, 3]|            [35, 18]|
|       4|  [4, 5, 12, 47, 17]|             [9, 22]|
|       5|      [18, 42, 5, 6]|     [33, 3, 35, 22]|
|       6|       [2, 4, 34, 6]|          [9, 46, 0]|
|       7| [7, 40, 15, 16, 23]|    [45, 37, 11, 32]|
|       8|   [2, 3, 34, 8, 29]|     [35, 18, 46, 0]|
|       9|[35, 36, 9, 18, 2...|              [3, 4]|
|      10|        [10, 44, 45]|[14, 7, 32, 37, 1...|
+--------+--------------------+--------------------+
only showing top 10 rows



## --- Use Flavor and Food

In [20]:
# Load dataset goods.csv
product_data = spark.read.csv('data/75000/goods.csv',
                             header=True,
                             inferSchema=True)

In [21]:
product_data.show(5, False)

+---+------------+------+-----+------+
|Id |Flavor      |Food  |Price|Type  |
+---+------------+------+-----+------+
|0  |'Chocolate' |'Cake'|8.95 |'Food'|
|1  |'Lemon'     |'Cake'|8.95 |'Food'|
|2  |'Casino'    |'Cake'|15.95|'Food'|
|3  |'Opera'     |'Cake'|15.95|'Food'|
|4  |'Strawberry'|'Cake'|11.95|'Food'|
+---+------------+------+-----+------+
only showing top 5 rows



In [22]:
product_data.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Flavor: string (nullable = true)
 |-- Food: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Type: string (nullable = true)



In [23]:
# Combine 2 columns: Flavor, Food
product_data = product_data.select('Id',concat(col("Flavor"), lit(" "),col("Food")).alias('Food_n_Flavor'))

In [24]:
product_data.show(5, False)

+---+-------------------+
|Id |Food_n_Flavor      |
+---+-------------------+
|0  |'Chocolate' 'Cake' |
|1  |'Lemon' 'Cake'     |
|2  |'Casino' 'Cake'    |
|3  |'Opera' 'Cake'     |
|4  |'Strawberry' 'Cake'|
+---+-------------------+
only showing top 5 rows



In [25]:
# Remove " ' " in column Food_n_Flavor
product_data = product_data.select('Id', regexp_replace(col('Food_n_Flavor'),"'","").alias('Food_n_Flavor'))

In [26]:
product_data.show(5, False)

+---+---------------+
|Id |Food_n_Flavor  |
+---+---------------+
|0  |Chocolate Cake |
|1  |Lemon Cake     |
|2  |Casino Cake    |
|3  |Opera Cake     |
|4  |Strawberry Cake|
+---+---------------+
only showing top 5 rows



In [27]:
# Explode column 'items'
data = data.select('order_id', explode(col('items')).alias('product_id'))

In [28]:
data.show(5)

+--------+----------+
|order_id|product_id|
+--------+----------+
|       1|         1|
|       1|        11|
|       1|        21|
|       2|         2|
|       2|        37|
+--------+----------+
only showing top 5 rows



In [29]:
# create sql view 'products' from product_data
product_data.createOrReplaceTempView('products')

In [30]:
# create sql view 'orders' from data
data.createOrReplaceTempView('orders')

In [32]:
# Join 2 views: products, orders by product_id / Id
query = 'select p.Id, p.Food_n_Flavor, o.order_id from products p inner join orders o where o.product_id=p.Id'
rawData = spark.sql(query)
baskets = rawData.groupBy('order_id').agg(collect_set('Food_n_Flavor').alias('Food_n_Flavor'))
baskets.createOrReplaceTempView('baskets')

In [33]:
baskets.show(3, False)

+--------+-------------------------------------------------------------------------------------------------------------------+
|order_id|Food_n_Flavor                                                                                                      |
+--------+-------------------------------------------------------------------------------------------------------------------+
|1       |[Ganache Cookie, Lemon Cake, Apple Pie]                                                                            |
|12      |[Almond Twist, Ganache Cookie, Opera Cake, Single Espresso, Casino Cake, Raspberry Lemonade, Apple Pie, Apple Tart]|
|22      |[Coffee Eclair, Gongolais Cookie, Blackberry Tart, Single Espresso]                                                |
+--------+-------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [34]:
# Build model and fit
fpGrowth = FPGrowth(itemsCol='Food_n_Flavor', minSupport=0.03, minConfidence=0.03)
model_1 = fpGrowth.fit(baskets)

In [35]:
# Get frequent items
model_1.freqItemsets.sort('freq', ascending=False).show(10)

+------------------+----+
|             items|freq|
+------------------+----+
|   [Coffee Eclair]|8194|
|      [Hot Coffee]|7701|
|    [Tuile Cookie]|7557|
|     [Cherry Tart]|6988|
| [Strawberry Cake]|6949|
|  [Apricot Danish]|6944|
|    [Orange Juice]|6872|
|[Gongolais Cookie]|6784|
| [Marzipan Cookie]|6734|
|      [Berry Tart]|6363|
+------------------+----+
only showing top 10 rows



In [36]:
# Predictions
mostPopularItemInABasket = model_1.transform(baskets)

In [37]:
mostPopularItemInABasket.show(3, False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------
 order_id      | 1                                                                                                                   
 Food_n_Flavor | [Ganache Cookie, Lemon Cake, Apple Pie]                                                                             
 prediction    | [Lemon Tart, Almond Twist, Hot Coffee, Coffee Eclair]                                                               
-RECORD 1----------------------------------------------------------------------------------------------------------------------------
 order_id      | 12                                                                                                                  
 Food_n_Flavor | [Almond Twist, Ganache Cookie, Opera Cake, Single Espresso, Casino Cake, Raspberry Lemonade, Apple Pie, Apple Tart] 
 prediction    | [Hot Coffee, Coffee Eclair, Chocolate Coffee,